In [4]:
!pip -q install redis redis-cli

In [2]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)

## Redis Strings Explained - [Link to Video](https://youtu.be/n0LQREq4GrY)

Redis strings are the most basic and common Redis data type, but they are also powerful. Strings in Redis are binary-safe sequences of bytes, similar to strings in most programming languages. You can store many types of data, including integers, binary values, comma-separated values, serialized JSON, and even larger objects like images, videos, documents, or sound files.

### Practical Uses

The most common use of Redis strings is caching, such as:

- API responses
- Session storage
- HTML pages

Strings are also useful for counters, with built-in support for incrementing and decrementing integers and floating-point values.

### Storing and Retrieving Strings

To create a string, use the `SET` command with a key and value. Example: `SET user:101:time-zone UTC+8`

### Command Breakdown

> **Sidenote**  
> **Command**: `SET`  
> Assigns a value to a key in Redis.  
>   
> **Key**: `user:101:time-zone`  
> - `user` → namespace prefix for user data  
> - `101` → unique user ID  
> - `time-zone` → field/attribute for this user  
>   
> **Pattern:** `object:id:field`  
>   
> **Value**: `UTC+8`  
> String value stored for the key (user’s time zone).  
>   
> **Result**:  
> ```
> Key: user:101:time-zone  
> Value: UTC+8  
> ```


In response, Redis replies with `OK`, confirming the value was stored. To retrieve it, use the `GET` command: `GET user:101:time-zone`


In [ ]:
''' 
Using redis-cli:
'''

!redis-cli -h 127.0.0.1 -p 6379 SET user:101:time-zone UTC+8
!redis-cli -h 127.0.0.1 -p 6379 GET user:101:time-zone

OK
"UTC+8"


In [ ]:
''' 
Using Python (API):
'''

r.set("user:101:time-zone", "UTC+8")
val = r.get("user:101:time-zone")
val.decode()

'UTC+8'

In [ ]:
''' 
Using Python (Raw Redis Command):
'''

r.execute_command("SET", "user:101:time-zone", "UTC+8")
print(r.execute_command("GET", "user:101:time-zone").decode())


UTC+8


### Caching JSON Objects

Strings can store JSON responses, which is helpful when caching results that don’t change often. For example, instead of repeatedly querying a data warehouse, cache the response under a key like `usage:63`.

When caching, you can set an expiration time with the `EX` option: `SET usage:63 "{...JSON data...}" EX 7200`

This sets the key to expire in 7,200 seconds (two hours). You can check remaining time with the `TTL` command.

In [10]:
r.execute_command("SET", "usage:63", "{...JSON data...}", "EX", 7200)

True

In [12]:
print(r.ttl("usage:63")) #seconds
print(r.pttl("usage:63")) #milliseconds
# Returns remaining time. -1 = no expiry. -2 = key missing.

7142
7142349


### Incrementing and Decrementing

Redis strings can also be manipulated numerically.

- `INCR key` increments the value by 1.
- `INCRBY key N` increments the value by N.
- Using a negative number decrements the value.
- If the key does not exist, Redis creates it automatically.

In [13]:
# increment visit count by 1 (creates key with 1 if not exist)
r.execute_command("INCR", "user:23:visit-count")
print("visit-count =", r.get("user:23:visit-count").decode())

# set credit balance to 40 (overwrite if exists)
r.execute_command("SET", "user:23:credit-balance", 40)
print("credit-balance =", r.get("user:23:credit-balance").decode())

# add 30 to credit balance (40 → 70)
r.execute_command("INCRBY", "user:23:credit-balance", 30)
print("credit-balance =", r.get("user:23:credit-balance").decode())

# subtract 50 from credit balance (70 → 20)
r.execute_command("INCRBY", "user:23:credit-balance", -50)
print("credit-balance =", r.get("user:23:credit-balance").decode())

visit-count = 1
credit-balance = 40
credit-balance = 70
credit-balance = 20


### Key Takeaways

- Redis strings are the fundamental data type.
- They can store text, numbers, JSON, or binary data.
- Strings with expirations are ideal for caching.
- They support built-in increment and decrement operations.

### Original Transcript
Video title: Redis Strings Explained
    
Video URL: https://youtu.be/n0LQREq4GrY
    
Video language: English (United States)
    
--------------------------------

Hello. Today we're going to explore the string, the most basic and common Redis date type. We'll see just how powerful this little guy can be. [MUSIC PLAYING]  Redis strings are binary safe sequences of bytes. And they're a lot like the strings you use in your favorite programming language. You can store almost anything you can imagine in a Redis string. Integers, binary values, comma-separated values, serialized JSON. And since they're binary safe, you can even store larger objects, such as images, videos, documents and sound. So what are some practical uses for Redis strings? The most common use is for caching: API responses, session storage, HTML pages. The opportunities are vast. Redis strings are also great for implementing counters, as there is built in support for incrementing and decrementing integer and floating point values. Today, we're going to go over a couple examples using the Redis string. First, we'll cover how to store and retrieve a string. Second, we'll examine a use case of storing a JSON object as a string, and how to set an expiration time until it's deletion. We'll also discuss the implications of storing JSON as a string, and a couple of alternatives within Redis. Lastly, we'll learn how to increment and decrement an integer value within a Redis string. To create a string in Redis, you'll use the SET command. After the SET keyword, you specify the key, followed by the value you want to associate with it. For example, let's create a time zone entry for a user of an application that we're building. I'll use user:101:time-zone as the key and UTC 8 as the value. Redis returns OK, which means that the time zone string was successfully stored at key user:101:time-zone. To retrieve a Redis string, you use the GET command, followed by the key you want to retrieve. Let's now get the user's time zone. Redis returns the value of the key as a string. Let's consider an example use of strings. Imagine you want to provide your user with site usage data. Normally, you'd send the request to a data warehouse, which might take several seconds to complete. But by using a string, you can cache this JSON response after initially fetching it. As these responses don't change often. Here we have a sample JSON response containing a user's recent site usage request. Instead of accessing our data warehouse for this information on subsequent requests, we can cache this response as a string within Redis. Let's assign this information to the key usage:63. When caching data, it's common to set an expiry time. Here we'll do that by adding the EX option to our SET command, followed by a number of seconds. EX specifies a time in seconds, after which Redis will expire and delete the key and its associated string value. So this Redis string will now expire in 7,200 seconds, or two hours. By using Redis as a cache, subsequent responses that might have taken several seconds to fetch from the data warehouse, will instead be served instantly. We can check on the remaining time to live for a key by using the TTL command. TTL returns the number of seconds remaining before the key expires. Now, expiry is a complex enough topic to warrant its own video segment. So check back soon for a dedicated video. Lastly, let's check out the abilities of the Redis string in regards to integer manipulation. With the INCR and INCRBY commands, you can increment a key's value by one or by a specified number. You can also use a negative number, which will decrement the value. If there is no key already present. The INCR and INCRBY commands will create a new string, and increment its value appropriately. To see how that works? Let's run the INCR command with a key that doesn't exist yet. Since the key user:23:visit-count doesn't exist yet, the INCR command simply creates a new key and string with a value of 1. This is the Redis way. If a data structure doesn't exist, then it will be created the moment you write to it. Now, let's take a quick look at the INCRBY command. INCRBY increments the value of a given key by a specified number we provide in the command line. Let's say we have a key user:23:credit-balance with a value 40. Let's increase the value to 70 with command INCRBY user:23:credit-balance 30. Finally, let's reflect the usage of the available credits for this user by running INCRBY with a negative number. Let's say our user consumes 50 credits. The command would be INCRBY user:colon:23:credit balance -50.  Now, the user has 20 credits left. OK. So let's do a quick review of what we just covered for Redis strings. Strings are the fundamental Redis data type, and you can use them to store plain old strings, numerical values, serialized JSON, and anything that can be represented in binary. Strings, combined with key expiry, are your first go to data structure for caching. Finally, you can also use Redis strings to store and manipulate numerical values. To learn more about Redis strings, check our free online course: Introduction to Redis Data Structures. It's part of Redis University, our free online learning platform for all things Redis. Thanks for joining me in this quick dive into Redis strings. Hope to see you again, and happy learning. [MUSIC PLAYING]


DISCLAIMER:


By using this tool to convert YouTube videos to Text, you acknowledge and agree to the following:


1. User Responsibility: You are responsible for ensuring that your use of this tool complies with all applicable copyright laws and YouTube's Terms of Use. This includes obtaining any necessary permissions from the original content creators before reproducing or distributing any content.


2. Content Ownership: The transcripts generated by this tool are based on publicly available content from YouTube. You do not own the rights to the original video content or its transcripts. All copyrights and ownership rights remain with the original content creators.


3. Attribution: This tool provides the URL of the original YouTube video for reference purposes. You are encouraged to include proper attribution when sharing the generated Text.


4. Limitations of Liability: We disclaim any liability for the misuse of the content generated through this tool. By using this tool, you agree to hold us harmless from any claims or disputes arising from your use of the content.